In [1]:
import os


In [2]:
%pwd

'c:\\Users\\diban\\Documents\\End-end ML project with MLFlow\\End-end-Projects-with-MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\diban\\Documents\\End-end ML project with MLFlow\\End-end-Projects-with-MLFlow'

In [5]:
#Tracking Url for the project
import dagshub
dagshub.init(repo_owner='Dibanisaf', repo_name='End-end-Projects-with-MLFlow-Wine-Quality-', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [6]:
import os
os.environ['MLFOW_TRACKING_USERNAME']= 'Dibanisar'
os.environ['MLFOW_TRACKING_PASSWORD']= '2b045b6956a9767ce2cd1165d59bf6e9c1fdb203'
os.environ['MLFLOW_TRACKING_URL'] = 'https://dagshub.com/Dibanisaf/End-end-Projects-with-MLFlow-Wine-Quality-.mlflow'

#mlflow.set_experiment("Wine-Quality")


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [8]:
from src.ML_Project.constants import *
from src.ML_Project.utils.common import read_yaml, create_directories,save_json


In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH
                     ,params_filepath = PARAMS_FILE_PATH
                         ,schema_filepath = SCHEMA_FILE_PATH ):
        
        self.config = read_yaml(config_filepath)
        self.params =read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root]) #creating the artifact_root path
        
        
    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config =self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        
        model_evaluation_config = ModelEvaluationConfig(
                root_dir = config.root_dir,
                test_data_path= config.test_data_path,
                model_path= config.model_path,
                all_params= params,
                metric_file_name= config.metric_file_name,
                target_column= schema.name,
                mlflow_uri= 'https://dagshub.com/Dibanisaf/End-end-Projects-with-MLFlow-Wine-Quality-.mlflow'

            
        )
        
        return model_evaluation_config

In [10]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import mlflow
from urllib.parse import urlparse
import mlflow.sklearn
import numpy as np
import joblib

In [11]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self,actual,pred):
        rsme = np.sqrt(mean_squared_error(actual,pred))
        mae =mean_absolute_error(actual,pred)
        r2 =r2_score(actual,pred)
        
        return rsme,mae,r2
    def log_into_mlflow(self):
        # Load the test data
        test_data = pd.read_csv(self.config.test_data_path)

        # Load the trained model
        model = joblib.load(self.config.model_path)

        # Split data into features (X) and target (y)
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Set the MLflow registry URI
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Start an MLflow run
        with mlflow.start_run():
            
            # mlflow.log_param('parameter name', 'value')
            # mlflow.log_metric('metric name', 1)
            # Make predictions
            predicted_qualities = model.predict(test_x)

            # Evaluate metrics
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            scores ={"rmse":rmse, "mae":mae, "r2":r2}
            
            save_json(path=Path(self.config.metric_file_name),data =scores)
            
            mlflow.log_params(self.config.all_params)

            # Log metrics (not visible in the screenshot but likely part of the code)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            
            
            if tracking_url_type_store != "file":
                #Note to self (Refer this part to the documentation it depends on the use case)!!!!!!!!!!!!!
                
                mlflow.sklearn.log_model(model,"model",registered_model_name= "ElasticnetModel")
                
            else:
                mlflow.sklearn.log_model(model,"model")
                
            

In [12]:
try:
    config =ConfigurationManager()
    model_evaluation_config =config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config= model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
    
except Exception as e:
    raise e


[2025-01-29 11:13:31,250:INFO:yaml file :config\config.yaml loaded successfully]
[2025-01-29 11:13:31,267:INFO:yaml file :params.yaml loaded successfully]
[2025-01-29 11:13:31,268:INFO:yaml file :schema.yaml loaded successfully]
[2025-01-29 11:13:31,273:INFO:created directory at: artifacts]
[2025-01-29 11:13:31,275:INFO:created directory at: artifacts/model_evaluation]
[2025-01-29 11:13:31,693:INFO:json file saved at :artifacts\model_evaluation\metrics.json]


2025/01/29 11:13:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
